In [1]:
from calendar import monthrange
from datetime import datetime
import pandas as pd
from pandas.api.types import CategoricalDtype
from io import BytesIO
import os
import json

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

# Load locale custome modules
import sys
sys.path.append(os.path.abspath("../tools"))
from CloudIO.AzStorageClient import AzStorageClient
from edahelpers import *

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', None)

In [2]:
root_dir =  os.path.abspath(os.getcwd()+"..\\..\\..\\..\\")
tmp_dir = root_dir + "\\.tmp"
data_dir = root_dir + "\\data\\"
cred_dir = root_dir + "\\.cred"
az_cred_file = cred_dir + '\\azblobcred.json'

site_metadata_filename = data_dir + "site-metadata.csv"

# Output
container = "gold-samples-data"
# blob_name = "gold_samples_full_data.parquet"
blob_name = "gold_samples_trim_data_0206.parquet" #Advisor suggested features only

In [3]:
# "Golden" Sites
tier1_sites = ["US-MMS", "US-Vcp", "FR-Pue", "CH-Lae", "US-Var", "US-Ne2", "ES-LJu", "US-Ton"]
tier2_sites = ["US-UMB", "US-Me2", "FI-Hyy", "US-NR1", "IT-Lav", "US-Wkg", "US-ARM", "US-SRM"]

target_sites = tier1_sites + tier2_sites

# Get Gold Sample Site Data

In [4]:
# Load site metadata
included_site_features = ['site_id', 'filename','IGBP', 'elevation', 'lat', 'long',
                          'koppen_sub', 'koppen_main', 'koppen_name', 'koppen_main_name',
                          'c3c4', 'c4_percent']
site_metadata_df = pd.read_csv(site_metadata_filename, usecols = included_site_features)

# only focus on target sites
site_metadata_df= site_metadata_df.loc[site_metadata_df['site_id'].isin(target_sites)]
print(f"size:{site_metadata_df.shape}")
site_metadata_df.reset_index(inplace=True, drop=True)
site_metadata_df

size:(16, 12)


,site_id,IGBP,elevation,lat,long,koppen_sub,koppen_main,koppen_name,koppen_main_name,c3c4,c4_percent,filename
0,FR-Pue,EBF,270.0,43.74130,3.59570,8,3,Csa,Temperate,C3,6.59,data_full_half_hourly_raw_v0_1_FR-Pue.csv
1,US-NR1,ENF,3050.0,40.03290,-105.54640,27,4,Dfc,Cold,C3,0.35,data_full_half_hourly_raw_v0_1_US-NR1.csv
2,US-Ne2,CRO,362.0,41.16487,-96.47010,25,4,Dfa,Cold,rotation,48.91,NaN
3,US-SRM,WSA,1120.0,31.82140,-110.86610,6,2,BSh,Arid,C3,55.39,data_full_half_hourly_raw_v0_1_US-SRM.csv
4,US-Ton,WSA,177.0,38.43160,-120.96598,8,3,Csa,Temperate,C3,0.00,data_full_half_hourly_raw_v0_1_US-Ton.csv
5,US-Var,GRA,129.0,38.41330,-120.95070,8,3,Csa,Temperate,C3,0.00,data_full_half_hourly_raw_v0_1_US-Var.csv
6,US-Wkg,GRA,1531.0,31.73650,-109.94190,7,2,BSk,Arid,C4,40.93,data_full_half_hourly_raw_v0_1_US-Wkg.csv
7,US-ARM,CRO,314.0,36.60580,-97.48880,14,3,Cfa,Temperate,mix,15.97,data_full_half_hourly_raw_v0_1_US-ARM.csv
8,US-MMS,DBF,275.0,39.32320,-86.41310,25,4,Dfa,Cold,C3,42.28,NaN
9,US-Me2,ENF,1253.0,44.45230,-121.55740,18,4,Dsb,Cold,C3,0.03,data_full_half_hourly_raw_v0_1_US-Me2.csv


In [5]:
all_features = ['TIMESTAMP_START', 'TIMESTAMP_END', 'TA_F', 'TA_F_QC', 'TA_ERA',
       'SW_IN_POT', 'SW_IN_F', 'SW_IN_F_QC', 'SW_IN_ERA', 'LW_IN_F',
       'LW_IN_F_QC', 'LW_IN_ERA', 'VPD_F', 'VPD_F_QC', 'VPD_ERA', 'P_F',
       'P_F_QC', 'P_ERA', 'PA_F', 'PA_F_QC', 'PA_ERA', 'NETRAD', 'PPFD_IN',
       'G_F_MDS', 'G_F_MDS_QC', 'LE_F_MDS', 'LE_F_MDS_QC', 'LE_CORR',
       'H_F_MDS', 'H_F_MDS_QC', 'H_CORR', 'NEE_VUT_REF', 'NEE_VUT_REF_QC',
       'NEE_CUT_REF', 'NEE_CUT_REF_QC', 'GPP_NT_VUT_REF', 'GPP_DT_VUT_REF',
       'GPP_NT_CUT_REF', 'GPP_DT_CUT_REF', 'RECO_NT_VUT_REF',
       'RECO_DT_VUT_REF', 'RECO_NT_CUT_REF', 'RECO_DT_CUT_REF', 'datetime',
       'year', 'month', 'day', 'hour', 'SITE_ID', 'date', 'NEE_VUT_REF_qa',
       'SW_DIF', 'EVI', 'NDVI', 'NIRv', 'b1', 'b2', 'b3', 'b4', 'b5', 'b6',
       'b7', 'IGBP', 'koppen']
included_features = ['TA_ERA', 'SW_IN_ERA', 'LW_IN_ERA', 'VPD_ERA', 'P_ERA', 'PA_ERA',
                     'NETRAD', 'PPFD_IN', 'NEE_VUT_REF', 'NEE_VUT_REF_QC',
                     'datetime', 'year', 'month', 'day', 'hour', 'date',
                     'EVI', 'NDVI', 'NIRv', 'b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 
                     'IGBP', 'koppen']
target_variable = ['GPP_NT_VUT_REF']
qc_flag_dtype = CategoricalDtype([0, 1, 2, 3], ordered=True)
qc_flags_features = [s for s in included_features if "_QC" in s]

In [6]:
# Read files
data_df = None
for i, r in site_metadata_df[['site_id','filename']].iterrows():        
    if not r.filename or type(r.filename) != type(""):
        print(f'\nERROR: {r.site_id} is mssing hourly data.')
        continue
    
    # Get only `included_features` from file
    local_filename = tmp_dir + "\\" + r.filename
    site_df = pd.read_csv(local_filename, usecols = included_features +  target_variable)
    site_df['datetime'] = pd.to_datetime(site_df['datetime'])
    site_df['date'] = pd.to_datetime(site_df['date'])
    site_df['minute'] = site_df['datetime'].dt.minute
    site_df[qc_flags_features] = site_df[qc_flags_features].astype(qc_flag_dtype)
    site_df['site_id'] = r.site_id
    
    # Remove zero or negative SW
    site_df.drop(site_df[site_df['SW_IN_ERA'] <= 0].index, inplace = True)
    
    # Drop rows with NAs for Target Variable
    site_df.dropna(subset=target_variable, axis='index', inplace=True)
    
    #TODO: Drop rows with bad QC flags for GPP?
    site_df.drop(['NEE_VUT_REF', 'NEE_VUT_REF_QC'], axis=1, inplace=True)
    
    print(f"{r.site_id}: {site_df.shape}")
    if type(data_df) == type(None):
        data_df = site_df
    else:
        data_df = pd.concat([data_df, site_df])

FR-Pue: (123299, 29)
US-NR1: (135937, 29)

ERROR: US-Ne2 is mssing hourly data.
US-SRM: (95756, 29)
US-Ton: (115546, 29)
US-Var: (122872, 29)
US-Wkg: (93607, 29)
US-ARM: (129363, 29)

ERROR: US-MMS is mssing hourly data.
US-Me2: (109417, 29)
US-UMB: (96484, 29)
US-Vcp: (87395, 29)
CH-Lae: (153502, 29)
ES-LJu: (117181, 29)
FI-Hyy: (216706, 29)
IT-Lav: (158858, 29)


In [7]:
data_df.head()

,TA_ERA,SW_IN_ERA,LW_IN_ERA,VPD_ERA,P_ERA,PA_ERA,NETRAD,PPFD_IN,GPP_NT_VUT_REF,datetime,year,month,day,hour,date,EVI,NDVI,NIRv,b1,b2,b3,b4,b5,b6,b7,IGBP,koppen,minute,site_id
11,16.848,18.478,336.152,3.049,0.0,97.965,-27.270,12.08,-0.437716,2000-07-26 05:30:00,2000,7,26,5,2000-07-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EBF,Temperate,30,FR-Pue
12,16.932,65.997,336.152,3.134,0.0,97.981,-16.690,74.61,0.879290,2000-07-26 06:00:00,2000,7,26,6,2000-07-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EBF,Temperate,0,FR-Pue
13,17.016,114.551,336.152,3.220,0.0,97.996,27.455,88.21,5.448740,2000-07-26 06:30:00,2000,7,26,6,2000-07-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EBF,Temperate,30,FR-Pue
14,17.100,252.095,339.113,3.305,0.0,98.012,84.205,274.01,6.434920,2000-07-26 07:00:00,2000,7,26,7,2000-07-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EBF,Temperate,0,FR-Pue
15,18.055,326.385,339.113,5.097,0.0,98.029,146.205,387.91,5.707650,2000-07-26 07:30:00,2000,7,26,7,2000-07-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EBF,Temperate,30,FR-Pue


In [8]:
data_df.describe()

,TA_ERA,SW_IN_ERA,LW_IN_ERA,VPD_ERA,P_ERA,PA_ERA,NETRAD,PPFD_IN,GPP_NT_VUT_REF,year,month,day,hour,EVI,NDVI,NIRv,b1,b2,b3,b4,b5,b6,b7,minute
count,1.704611e+06,1.704611e+06,1.704611e+06,1.704611e+06,1.704611e+06,1.704611e+06,1.409825e+06,1.694537e+06,1.755923e+06,1.755923e+06,1.755923e+06,1.755923e+06,1.755923e+06,1.583379e+06,1.585494e+06,1.585494e+06,1.585536e+06,1.586560e+06,1.584243e+06,1.585482e+06,1.582550e+06,1.536679e+06,1.586253e+06,1.755923e+06
mean,1.351667e+01,3.580911e+02,3.122694e+02,9.249479e+00,3.987029e-02,9.072458e+01,2.293942e+02,6.985750e+02,5.581373e+00,2.009758e+03,6.485680e+00,1.573964e+01,1.187538e+01,3.008528e-01,5.437718e-01,1.316936e-01,7.781695e-02,2.408251e-01,4.321027e-02,7.074933e-02,2.549803e-01,1.981325e-01,1.153479e-01,1.499975e+01
std,9.662309e+00,2.662761e+02,4.838676e+01,9.361774e+00,1.814388e-01,9.152955e+00,2.254630e+02,5.833798e+02,7.489330e+00,5.154447e+00,3.138760e+00,8.810634e+00,3.865977e+00,1.344684e-01,2.197028e-01,7.177069e-02,5.632201e-02,6.184810e-02,4.150399e-02,4.346184e-02,7.027749e-02,9.334317e-02,7.816978e-02,1.500000e+01
min,-2.974000e+01,1.000000e-03,1.427700e+02,0.000000e+00,0.000000e+00,6.728100e+01,-3.652033e+02,-9.140000e+01,-4.973720e+01,1.996000e+03,1.000000e+00,1.000000e+00,0.000000e+00,-1.195764e-01,-1.825229e-01,-1.715080e-02,0.000000e+00,3.050000e-02,0.000000e+00,0.000000e+00,0.000000e+00,1.320000e-02,0.000000e+00,0.000000e+00
25%,6.888000e+00,1.206770e+02,2.794880e+02,2.596000e+00,0.000000e+00,8.523700e+01,3.357360e+01,1.702000e+02,3.424810e-01,2.006000e+03,4.000000e+00,8.000000e+00,9.000000e+00,2.076478e-01,3.464472e-01,8.228100e-02,3.250000e-02,1.975000e-01,1.900000e-02,4.130000e-02,1.993000e-01,1.179000e-01,4.970000e-02,0.000000e+00
50%,1.373700e+01,3.171270e+02,3.137750e+02,6.070000e+00,0.000000e+00,9.331200e+01,1.793000e+02,5.639530e+02,2.867450e+00,2.010000e+03,6.000000e+00,1.600000e+01,1.200000e+01,2.901429e-01,5.772830e-01,1.192774e-01,6.180000e-02,2.300000e-01,3.310000e-02,6.180000e-02,2.565000e-01,1.824000e-01,9.440000e-02,0.000000e+00
75%,2.034100e+01,5.615975e+02,3.462925e+02,1.252100e+01,0.000000e+00,9.873700e+01,3.961500e+02,1.135780e+03,9.059005e+00,2.014000e+03,9.000000e+00,2.300000e+01,1.500000e+01,3.622077e-01,7.126663e-01,1.599520e-01,1.152000e-01,2.771000e-01,5.440000e-02,9.080000e-02,3.132000e-01,2.761000e-01,1.786000e-01,3.000000e+01
max,4.258700e+01,1.094341e+03,4.730110e+02,7.568400e+01,1.549300e+01,1.033830e+02,1.035366e+03,2.527130e+03,8.503090e+01,2.020000e+03,1.200000e+01,3.100000e+01,2.300000e+01,2.388354e+00,1.000000e+00,4.238474e-01,7.971000e-01,7.849000e-01,7.689000e-01,7.865000e-01,4.666000e-01,4.280000e-01,3.573000e-01,3.000000e+01


In [9]:
data_df.shape

(1755923, 29)

In [10]:
data_df.site_id.unique()

array(['FR-Pue', 'US-NR1', 'US-SRM', 'US-Ton', 'US-Var', 'US-Wkg',
       'US-ARM', 'US-Me2', 'US-UMB', 'US-Vcp', 'CH-Lae', 'ES-LJu',
       'FI-Hyy', 'IT-Lav'], dtype=object)

## Merge with Site Metadata

In [11]:
data_df = data_df.merge(site_metadata_df.drop(['filename'], axis=1), how='left', left_on='site_id', right_on='site_id')
data_df.head()

,TA_ERA,SW_IN_ERA,LW_IN_ERA,VPD_ERA,P_ERA,PA_ERA,NETRAD,PPFD_IN,GPP_NT_VUT_REF,datetime,year,month,day,hour,date,EVI,NDVI,NIRv,b1,b2,b3,b4,b5,b6,b7,IGBP_x,koppen,minute,site_id,IGBP_y,elevation,lat,long,koppen_sub,koppen_main,koppen_name,koppen_main_name,c3c4,c4_percent
0,16.848,18.478,336.152,3.049,0.0,97.965,-27.270,12.08,-0.437716,2000-07-26 05:30:00,2000,7,26,5,2000-07-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EBF,Temperate,30,FR-Pue,EBF,270.0,43.7413,3.5957,8,3,Csa,Temperate,C3,6.59
1,16.932,65.997,336.152,3.134,0.0,97.981,-16.690,74.61,0.879290,2000-07-26 06:00:00,2000,7,26,6,2000-07-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EBF,Temperate,0,FR-Pue,EBF,270.0,43.7413,3.5957,8,3,Csa,Temperate,C3,6.59
2,17.016,114.551,336.152,3.220,0.0,97.996,27.455,88.21,5.448740,2000-07-26 06:30:00,2000,7,26,6,2000-07-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EBF,Temperate,30,FR-Pue,EBF,270.0,43.7413,3.5957,8,3,Csa,Temperate,C3,6.59
3,17.100,252.095,339.113,3.305,0.0,98.012,84.205,274.01,6.434920,2000-07-26 07:00:00,2000,7,26,7,2000-07-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EBF,Temperate,0,FR-Pue,EBF,270.0,43.7413,3.5957,8,3,Csa,Temperate,C3,6.59
4,18.055,326.385,339.113,5.097,0.0,98.029,146.205,387.91,5.707650,2000-07-26 07:30:00,2000,7,26,7,2000-07-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EBF,Temperate,30,FR-Pue,EBF,270.0,43.7413,3.5957,8,3,Csa,Temperate,C3,6.59


In [12]:
if data_df[data_df['IGBP_x'] != data_df['IGBP_y']].shape[0] != 0:
     raise Exception("Mismatched IGBP")
else:
    data_df.drop(['IGBP_y'], axis=1, inplace=True)
    data_df.rename(columns={'IGBP_x':'IGBP'}, inplace=True)
    display(data_df)

,TA_ERA,SW_IN_ERA,LW_IN_ERA,VPD_ERA,P_ERA,PA_ERA,NETRAD,PPFD_IN,GPP_NT_VUT_REF,datetime,year,month,day,hour,date,EVI,NDVI,NIRv,b1,b2,b3,b4,b5,b6,b7,IGBP,koppen,minute,site_id,elevation,lat,long,koppen_sub,koppen_main,koppen_name,koppen_main_name,c3c4,c4_percent
0,16.848,18.478,336.152,3.049,0.0,97.965,-27.270,12.080000,-0.437716,2000-07-26 05:30:00,2000,7,26,5,2000-07-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EBF,Temperate,30,FR-Pue,270.0,43.7413,3.59570,8,3,Csa,Temperate,C3,6.59
1,16.932,65.997,336.152,3.134,0.0,97.981,-16.690,74.610000,0.879290,2000-07-26 06:00:00,2000,7,26,6,2000-07-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EBF,Temperate,0,FR-Pue,270.0,43.7413,3.59570,8,3,Csa,Temperate,C3,6.59
2,17.016,114.551,336.152,3.220,0.0,97.996,27.455,88.210000,5.448740,2000-07-26 06:30:00,2000,7,26,6,2000-07-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EBF,Temperate,30,FR-Pue,270.0,43.7413,3.59570,8,3,Csa,Temperate,C3,6.59
3,17.100,252.095,339.113,3.305,0.0,98.012,84.205,274.010000,6.434920,2000-07-26 07:00:00,2000,7,26,7,2000-07-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EBF,Temperate,0,FR-Pue,270.0,43.7413,3.59570,8,3,Csa,Temperate,C3,6.59
4,18.055,326.385,339.113,5.097,0.0,98.029,146.205,387.910000,5.707650,2000-07-26 07:30:00,2000,7,26,7,2000-07-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EBF,Temperate,30,FR-Pue,270.0,43.7413,3.59570,8,3,Csa,Temperate,C3,6.59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1755918,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.253429,1.389670,2020-12-31 21:30:00,2020,12,31,21,2020-12-31,0.30255,0.263407,0.084396,0.1868,0.3204,0.1783,0.1821,0.0978,0.0305,0.016,ENF,Cold,30,IT-Lav,1353.0,45.9562,11.28132,26,4,Dfb,Cold,C3,3.57
1755919,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.249702,1.370190,2020-12-31 22:00:00,2020,12,31,22,2020-12-31,0.30255,0.263407,0.084396,0.1868,0.3204,0.1783,0.1821,0.0978,0.0305,0.016,ENF,Cold,0,IT-Lav,1353.0,45.9562,11.28132,26,4,Dfb,Cold,C3,3.57
1755920,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.257156,1.388810,2020-12-31 22:30:00,2020,12,31,22,2020-12-31,0.30255,0.263407,0.084396,0.1868,0.3204,0.1783,0.1821,0.0978,0.0305,0.016,ENF,Cold,30,IT-Lav,1353.0,45.9562,11.28132,26,4,Dfb,Cold,C3,3.57
1755921,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.253429,1.389050,2020-12-31 23:00:00,2020,12,31,23,2020-12-31,0.30255,0.263407,0.084396,0.1868,0.3204,0.1783,0.1821,0.0978,0.0305,0.016,ENF,Cold,0,IT-Lav,1353.0,45.9562,11.28132,26,4,Dfb,Cold,C3,3.57


## Upload Data to Azure Storage Blob as Parquet
**Run with Caution!!!**

In [13]:
# Upload to Azure Storage Blob
# ref: https://stackoverflow.com/a/54666079
parquet_file = BytesIO()
data_df.to_parquet(parquet_file, engine='pyarrow')
parquet_file.seek(0)

azStorageClient = AzStorageClient(az_cred_file)
azStorageClient.uploadBlob(container, blob_name, parquet_file, overwrite=True)

File uploaded to gold-samples-data/gold_samples_trim_data_0206.parquet


## View NA Data

In [16]:
total_record_count = data_df.shape[0]
na_df = pd.DataFrame(data_df.isna().sum())
na_df["percentage"] = (na_df / total_record_count)
na_df.rename(columns={0:"count"}, inplace=True)

In [17]:
na_df.loc[(na_df['count'] != 0)].sort_values("percentage", ascending=False)

,count,percentage
NETRAD,346098,0.197103
b6,219244,0.124860
b5,173373,0.098736
EVI,172544,0.098264
b3,171680,0.097772
b4,170441,0.097066
NDVI,170429,0.097059
NIRv,170429,0.097059
b1,170387,0.097036
b7,169670,0.096627


## Get NA Percentage per Sites

In [18]:
sites = data_df['site_id'].unique()
features =  data_df.columns
plot_data = pd.DataFrame(columns=features[:-1])

for i, s in enumerate(data_df['site_id'].unique()):

    site_df = data_df[data_df['site_id'] == s]
    
    site_na_df = pd.DataFrame(site_df.isna().sum())
    site_na_df["percentage"] = round( 100*(site_na_df / site_df.shape[0]), 2)
    site_na_df.drop(["site_id"], inplace=True)
    #display(site_na_df.T)
    site_na_df.drop([0], axis=1, inplace=True)
    
    plot_data.loc[s] = site_na_df["percentage"]

plot_data

,TA_ERA,SW_IN_ERA,LW_IN_ERA,VPD_ERA,P_ERA,PA_ERA,NETRAD,PPFD_IN,GPP_NT_VUT_REF,datetime,year,month,day,hour,date,EVI,NDVI,NIRv,b1,b2,b3,b4,b5,b6,b7,IGBP,koppen,minute,site_id,elevation,lat,long,koppen_sub,koppen_main,koppen_name,koppen_main_name,c3c4
FR-Pue,0.00,0.00,0.00,0.00,0.00,0.00,1.80,8.27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.75,3.71,3.71,3.71,3.71,3.75,3.71,4.10,4.31,3.74,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
US-NR1,0.00,0.00,0.00,0.00,0.00,0.00,0.73,0.80,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.58,20.51,20.51,20.51,20.51,20.58,20.54,20.73,27.11,20.51,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
US-SRM,0.00,0.00,0.00,0.00,0.00,0.00,1.40,0.39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.13,0.07,0.07,0.07,0.07,0.13,0.07,0.07,0.18,0.13,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
US-Ton,0.00,0.00,0.00,0.00,0.00,0.00,2.49,0.10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.80,0.80,0.80,0.80,0.80,0.80,0.80,0.83,1.99,0.80,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
US-Var,0.00,0.00,0.00,0.00,0.00,0.00,2.47,0.22,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.54,1.54,1.54,1.54,1.54,1.54,1.54,1.54,2.29,1.54,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
US-Wkg,0.00,0.00,0.00,0.00,0.00,0.00,0.59,0.11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.26,0.26,0.26,0.26,0.26,0.26,0.26,0.26,0.31,0.26,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
US-ARM,0.00,0.00,0.00,0.00,0.00,0.00,9.58,8.41,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.11,1.11,1.11,1.11,1.11,1.11,1.11,1.21,2.36,1.11,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
US-Me2,0.00,0.00,0.00,0.00,0.00,0.00,2.60,1.72,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.22,5.22,5.22,5.22,5.22,5.22,5.22,5.41,8.11,5.24,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
US-UMB,0.00,0.00,0.00,0.00,0.00,0.00,5.95,2.60,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.07,23.04,23.04,23.04,23.04,23.07,23.04,23.90,25.75,23.04,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
US-Vcp,0.00,0.00,0.00,0.00,0.00,0.00,30.42,7.59,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.58,5.55,5.55,5.55,5.52,5.58,5.55,6.22,9.11,5.52,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
plot_df = plot_data.loc[:, (plot_data.sum(axis=0) != 0)]
plot_df.shape

(14, 18)

In [21]:
fig = px.imshow(plot_df,
                color_continuous_scale = 'amp',
                text_auto=False,
                title= "Proportions of Missing Data per Feature and Site",
                labels=dict(x="Features", y="Site ID",
                            color="Missing<br>Data(%)"),
               )
fig.update(data=[{'hovertemplate': '%{y} (%{x})<br>Missing: %{z}%'}])
fig.update_layout(
    height=500, width=600,
    margin={"r":0,"t":75,"l":0,"b":0},
    title={'y':0.95,'x':0.025},
    coloraxis_colorbar=dict(
        title_font_size = 14,
        tickfont_size = 12,
        lenmode="pixels", len=300,
        thicknessmode="pixels", thickness=15
    )
)

fig.show()